# Downloading COVID Data From FloridaDisaster.ORG

### To Do
- [ ] Fix the problem where before doh reported 0 as 0 and not missing, I can't use the data
- [X] Extract the page range 12-25
- [X] Find the pages that contain the words "Coronavirus: All persons tested"
- [X] Extract just those two pages pdf 
- [X] Export individual files to txt for storage



In [9]:
from selenium import webdriver
import os
import wget
import PyPDF2 as pdf
from PyPDF2 import PdfFileReader, PdfFileWriter
import time
import textract
from pathlib import Path
import re
import pandas as pd
import datetime

In [3]:
chrome_path = r"chromedriver.exe"

## Initializing driver element and finding 

In [4]:
driver = webdriver.Chrome(chrome_path)
driver.get('https://floridadisaster.org/covid19/covid-19-data-reports/')
title = driver.title
time.sleep(3)
result = driver.find_elements_by_xpath("//a[@href]")

In [ ]:
print(result[220].text)

In [ ]:
# type(result)

In [ ]:
# print(len(result))

In [ ]:
print(result)

In [5]:
cwd = os.getcwd()

In [6]:
pdf_output_path = Path(f'{cwd}/Data/PDF')

In [ ]:
print(pdf_output_path)

## Downloads the PDF Files from the FDEM Website

In [8]:
link_counter = 0
len_result = len(result)
for links in result:
    if links.text.find('Data Report') > 0:
        link_url = links.get_attribute('href')
        filename = wget.filename_from_url(link_url)
        if not os.path.exists(f'{pdf_output_path}/{filename}'):
            wget.download(link_url, out = f'{pdf_output_path}')
        link_counter +=1
        print (f'{round((link_counter * 2)/len_result,3)*100}% Complete',end='\r')

 

### Extracting PDF text

In [ ]:
print(pdf_file)

In [10]:
county_list = ['Alachua', 'Dade','Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia', 'Dade', 'Desoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Santa Rosa', 'Sarasota', 'Seminole', 'St. Johns', 'St. Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Unknown', 'UNKOWN', 'Volusia', 'Wakulla', 'Walton', 'Washington', 
]

In [58]:

text_line_search = '\w+\s[\w,:]+\s.*\Z'#'^[\w+\s\D+].+\Z'
digit_search ='^\d.*\Z'
date_search = '^\w+\s\d.+'
#turn into a function
page_counter = 0

today = datetime.date.today()
for files in os.listdir(pdf_output_path):
    
    pdf_writer = PdfFileWriter()
    header = []
    print(files)
#     print(f'This is the start of the output for: {files}')
    headerset = 0
    data_row = []
    pdf_file = open(f'{pdf_output_path}/{files}','rb')
    pdfreader = PdfFileReader(pdf_file)
    num_pages = pdfreader.numPages
    if num_pages < 23:
        up_limit = num_pages
    else:
        up_limit = 23
    dfset =0
    county_added_list = []
    for pages in range(3, up_limit):
        
        datalines = 0
        page =  pdfreader.getPage(pages)

        if page.extractText().find('Coronavirus: PUI testing by county') > 0 or page.extractText().find('Coronavirus: All persons with tests reported') > 0:
            pdf_writer.addPage(page)
            page_text = page.extractText()
            col_num = 0
            index_num = 0
            for lines in page_text.split('\n'):
                line_n = len(page_text.split('\n'))
                write_line = 0
                if not re.match(text_line_search, lines, flags=0)and not re.match(date_search, lines, flags=0)and datalines == 0 and lines not in county_list and headerset !=1:
                    header.append(lines)         
                if re.match(date_search, lines, flags=0):
                    date = lines[0:12]
                if lines in county_list:
                    datalines = 1
                    data_row = []
                    col_num=0
                if datalines == 1:
                    for names in header:
                        if names == 'Awaiting ':
                            header[index_num + 1] = 'Awaiting Testing'
                            header.pop(index_num)
                        if names == 'Percent ':
                            header[index_num + 1] = 'Pecent Positive'
                            header.pop(index_num) 
                        index_num += 1
                    headerset = 1
                    if headerset == 1 and dfset == 0 and len(lines) <30:       
                        df = pd.DataFrame(columns=header)
                        dfset = 1
                    if (len(data_row) + 1) <= len(header) and len(lines) < 20:
                        if len(lines) == 0:
#                             data_row[col_num] = 0
#                             data_row.append('0')
                            print('')

                        else:
                            data_row.append(lines)
                    if datalines == 1 and len(lines) > 2 and not re.match(date_search, lines, flags=0) and not re.match(text_line_search, lines, flags=0):
                        write_line = 1                        
                    if len(data_row) == len(header):
                        if data_row[0] not in county_added_list:
                            data = pd.DataFrame([data_row], columns=header)
                            df = pd.concat([df, data])
                            county_added_list.append(data_row[0])
                col_num += 1
    df.to_csv(f'data/TXT2/Text files of {files}.txt', index=False, sep="\t")
    output = f'Data/Unclean PDF/Unclean Extract from {files} ({today})'
    with open(output, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)

#             print(df)
                                    

06062020state_reports_latest.pdf

06_13_2020state_reports_latest.pdf

covid-19-data---daily-report-2020-03-16-1815.pdf
covid-19-data---daily-report-2020-03-18-0955.pdf
covid-19-data---daily-report-2020-03-19-0954.pdf
covid-19-data---daily-report-2020-03-19-1653.pdf
covid-19-data---daily-report-2020-03-20-1740-modified.pdf
covid-19-data---daily-report-2020-03-21-1008.pdf
covid-19-data---daily-report-2020-03-21-1731.pdf
covid-19-data---daily-report-2020-03-22-0951.pdf
covid-19-data---daily-report-2020-03-22-1828.pdf
covid-19-data---daily-report-2020-03-23-0949.pdf
covid-19-data---daily-report-2020-03-23-1709.pdf
covid-19-data---daily-report-2020-03-24-1012.pdf
covid-19-data---daily-report-2020-03-24-1657.pdf
covid-19-data---daily-report-2020-03-25-1007.pdf

covid-19-data---daily-report-2020-03-25-1745.pdf
covid-19-data---daily-report-2020-03-26-1107.pdf
covid-19-data---daily-report-2020-03-26-1659.pdf
covid-19-data---daily-report-2020-03-27-1026.pdf
covid-19-data---daily-report-2020-03-2

In [ ]:
# data = pd.DataFrame([data_row],columns=header)
print(data)
# print(header)

Eventually you need to clean up the formating of the filenames. They are very sloppy coming out of DOH.

In [62]:
files_t = '06062020state_reports_latest.pdf'
date_format = ["r'^(\d{8})'","r'^([\d_]+)\w'"]
for date_types in date_format:
    print(date_types)
    output = re.findall(r'^(\d{8})', files_t, flags=0)[0]
    print(output)
#     report_date = f'{output[0]}{output[1]}-{output[2]}{output[3]}-{output[4]}{output[5]}{output[6]}{output[7]}'
print(report_date)
# print(report_date)
# if bool(re.search(r'^\d{8}',test, flags=0)) == True:
#     print('FUCK YEAR FUCKERS!!!!')

r'^(\d{8})'
06062020
r'\w'
06062020
06-06-2020


## Extracting a page range from the PDFs

### Steps:
- [X] Open PDFs
- [ ] Extract the page range 12-25
- [ ] Convert that to a word document
- [ ] Find the pages that contain the words "Coronavirus: All persons tested"
- [ ] Extract just those two pages from the Word Document or the 


In [ ]:
print(os.listdir(pdf_output_path))

In [ ]:

def ExtractPDF(input_path):
    import datetime
    today = datetime.date.today()
    pdf_writer = PdfFileWriter()
    
    for files in os.listdir(input_path):
        pdf_file = open(f'{input_path}/{files}','rb')
        print(files)
        print(pdf_file)
        
        print(f'This is the filename: {files}')
        pdfreader = PdfFileReader(pdf_file)
        num_pages = pdfreader.numPages
        for pages in range(12,26):
            page = pdfreader.getPage(pages)
#             print(page)
            pdf_writer.addPage(page)
        output = f'Data/Unclean PDF/Unclean Extract from {files} ({today})'
        with open(output, 'wb') as output_pdf:
            pdf_writer.write(output_pdf)
        break
ExtractPDF(pdf_output_path)

In [ ]:
driver.close()

In [ ]:
os.listdir(cwd)

In [ ]:
print(cwd)
os.chdir('Data/Unclean PDF')
